# Exercice about Question Generation
 Question Generation
We are going to use the squad dataset for question generation.


In [ ]:
#TODO: load squad dataset
%%capture
!pip install datasets


In [ ]:
from datasets import load_dataset

squad = load_dataset("squad")

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
squad
# noticed : pas les même longeurs

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
squad_train = load_dataset("squad", split="train")
squad_val = load_dataset("squad", split="validation")

In [ ]:
squad_train

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [ ]:
squad_val

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 10570
})

In [ ]:
#observation dataset_train
from datasets import ClassLabel
import numpy as np
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(squad, num_examples=10):
    assert num_examples <= len(squad)
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(squad)-1)
        while pick in picks:
            pick = random.randint(0, len(squad)-1)
        picks.append(pick)
    
    df = pd.DataFrame(squad[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(squad_train, num_examples=10)

,id,title,context,question,answers
0,56e7c53b00c9c71400d775a6,Tristan_da_Cunha,"On 23 May 2001, the islands experienced an extratropical cyclone that generated winds up to 190 kilometres per hour (120 mph). A number of structures were severely damaged and a large number of cattle were killed, prompting emergency aid, provided by the British government.",Who provided aid for the extratropical cyclone?,"{'text': ['the British government'], 'answer_start': [251]}"
1,5706a1ad75f01819005e7c7e,New_Delhi,"The India Gate built in 1931 was inspired by the Arc de Triomphe in Paris. It is the national monument of India commemorating the 90,000 soldiers of the Indian Army who lost their lives while fighting for the British Raj in World War I and the Third Anglo-Afghan War.",The India Gate commemorates the lost soldiers of what army?,"{'text': ['Indian Army'], 'answer_start': [153]}"
2,5727b0412ca10214002d93e2,On_the_Origin_of_Species,"Darwin proposes sexual selection, driven by competition between males for mates, to explain sexually dimorphic features such as lion manes, deer antlers, peacock tails, bird songs, and the bright plumage of some male birds. He analysed sexual selection more fully in The Descent of Man, and Selection in Relation to Sex (1871). Natural selection was expected to work very slowly in forming new species, but given the effectiveness of artificial selection, he could ""see no limit to the amount of change, to the beauty and infinite complexity of the coadaptations between all organic beings, one with another and with their physical conditions of life, which may be effected in the long course of time by nature's power of selection"". Using a tree diagram and calculations, he indicates the ""divergence of character"" from original species into new species and genera. He describes branches falling off as extinction occurred, while new branches formed in ""the great Tree of life ... with its ever branching and beautiful ramifications"".",What did Darwin expect to see between species with the addition of artificial selection?,"{'text': ['no limit to the amount of change, to the beauty and infinite complexity of the coadaptations between all organic beings'], 'answer_start': [470]}"
3,56fa0064f34c681400b0bf46,Somalis,"In antiquity, the ancestors of the Somali people were an important link in the Horn of Africa connecting the region's commerce with the rest of the ancient world. Somali sailors and merchants were the main suppliers of frankincense, myrrh and spices, items which were considered valuable luxuries by the Ancient Egyptians, Phoenicians, Mycenaeans and Babylonians.","Along with myrrh and spices, what important luxury did ancient Somali merchants provide?","{'text': ['frankincense'], 'answer_start': [219]}"
4,57262ff638643c19005ad2bf,Southeast_Asia,"Homo sapiens reached the region by around 45,000 years ago, having moved eastwards from the Indian subcontinent. Homo floresiensis also lived in the area up until 12,000 years ago, when they became extinct. Austronesian people, who form the majority of the modern population in Indonesia, Malaysia, Brunei, East Timor, and the Philippines, may have migrated to Southeast Asia from Taiwan. They arrived in Indonesia around 2000 BC,and as they spread through the archipelago, they often settled along coastal areas and confined indigenous peoples such as Negritos of the Philippines or Papuans of New Guinea to inland regions.",How many years ago did Homo sapiens reached the Southeast Asia region?,"{'text': ['45,000 years ago'], 'answer_start': [42]}"
5,56f8bc609b226e1400dd0ee3,Gene,"Transcription produces a single-stranded RNA molecule known as messenger RNA, whose nucleotide sequence is complementary to the DNA from which it was transcribed.:6.1 The mRNA acts as an intermediate between the DNA gene and its final protein product. The gene's DNA is used as a template to generate a complementary mRNA. The mRNA matches the sequence of the gene's DNA coding strand because it is s

## Data analysis

A method to differentiate between several types of questions


In [ ]:
#TODO: suggest a method to get what/how/why questions from squad data 
#Je choisis ici de réaliser les manipulations uniquement sur le dataset train pour aléger l'exécution, et aussi parce que dans cet exo on s'intéresse pas encore à l'étape de training

#objectif 
#1 - extraction 3 sous-dataframes contenant séparément chaque question et leur contexte.
#2 - peut-être : selsection des colonnes : answers, context, question ? 
#3 - dataframe to dictionnaire 

# 1 - dataset to dataframe 
df = pd.DataFrame(squad_train)

In [ ]:
# extraction des sous-dataframe à l'aide de .loc
#expace important à ajouter 
df_what_train=df.loc[df['question'].str.lower().str.contains('what ')]
df_how_train=df.loc[df['question'].str.lower().str.contains('how ')]
df_why_train=df.loc[df['question'].str.lower().str.contains('why ')]

In [ ]:
df_what_train.head()

,id,title,context,question,answers
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...
7,5733bf84d058e614000b61c0,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",What is the daily student paper at Notre Dame ...,"{'text': ['The Observer'], 'answer_start': [598]}"
9,5733bf84d058e614000b61c1,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",In what year did the student paper Common Sens...,"{'text': ['1987'], 'answer_start': [908]}"


In [ ]:
df_why_train.head()

,id,title,context,question,answers
614,56bfab98a10cfb1400551220,Beyoncé,"In August, the couple attended the 2011 MTV Vi...",Why was the broadcast the most-watched in hist...,"{'text': ['Her appearance'], 'answer_start': [..."
691,56d4d9392ccc5a1400d8329f,Beyoncé,Beyoncé's vocal range spans four octaves. Jody...,Why is Beyoncé known as the centerpiece of Des...,"{'text': ['Her vocal abilities'], 'answer_star..."
1153,573426864776f4190066197e,Montana,"Under Territorial Governor Thomas Meagher, Mon...",Why was this constitutional convention held?,"{'text': ['bid for statehood'], 'answer_start'..."
1214,57321200e17f3d140042265d,Genocide,The study of genocide has mainly been focused ...,Why does genocide often go unpunished?,{'text': ['genocide is more often than not com...
1382,57328a33b3a91d1900202e29,Antibiotics,Antibiotics are screened for any negative effe...,Why are antibiotics checked before use?,{'text': ['negative effects on humans or other...


In [ ]:
df_how_train.head()

,id,title,context,question,answers
6,5733bf84d058e614000b61bf,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",How often is Notre Dame's the Juggler published?,"{'text': ['twice'], 'answer_start': [441]}"
8,5733bf84d058e614000b61bd,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",How many student news papers are found at Notr...,"{'text': ['three'], 'answer_start': [126]}"
15,5733a6424776f41900660f51,University_of_Notre_Dame,The College of Engineering was established in ...,How many BS level degrees are offered in the C...,"{'text': ['eight'], 'answer_start': [487]}"
18,5733a6424776f41900660f50,University_of_Notre_Dame,The College of Engineering was established in ...,How many departments are within the Stinson-Re...,"{'text': ['five'], 'answer_start': [271]}"
21,5733a70c4776f41900660f62,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...,How many colleges for undergraduates are at No...,"{'text': ['five'], 'answer_start': [68]}"


In [ ]:
# 2 - ne garder que les collonnes qui sont concernées par le sujet (input : "context", "answers" ; output : "question" )

df_what_clean = df_what_train.filter(items = ["question","context", "answers"])
df_how_clean = df_how_train.filter(items = ["question","context", "answers"])
df_why_clean = df_why_train.filter(items = ["question","context", "answers"])

In [ ]:
#3 - conversion dataframe to dictionnaire python pour pourvoir traiter les données

df_what_clean.dtypes

question    object
context     object
answers     object
dtype: object

In [ ]:
#colonne question comme key

dict_what = df_what_clean.set_index('question').T.to_dict()
dict_how = df_how_clean.set_index('question').T.to_dict()
dict_why = df_why_clean.set_index('question').T.to_dict()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """


In [ ]:
#deuxième option : zip 
d_what = dict([(i,[a,b]) for i,a,b in zip(df_what_clean['question'], df_what_clean['context'],df_what_clean['answers'])])
d_how = dict([(i,[a,b]) for i,a,b in zip(df_how_clean['question'], df_how_clean['context'],df_how_clean['answers'])])
d_why = dict([(i,[a,b]) for i,a,b in zip(df_why_clean['question'], df_why_clean['context'],df_why_clean['answers'])])

Consider this context from squad data:

"""
The Lophotrochozoa, evolved within Protostomia, include two of the most successful animal phyla, **the Mollusca and Annelida**. The former, which is the second-largest animal phylum by number of described species, includes animals such as snails, clams, and squids, and the latter comprises the segmented worms, such as earthworms and leeches. These two groups have long been considered close relatives because of the common presence of trochophore larvae, but the annelids were considered closer to the arthropods because they are both segmented. Now, this is generally considered convergent evolution, owing to many morphological and genetic differences between the two phyla. The Lophotrochozoa also include the Nemertea or ribbon worms, the Sipuncula, and several phyla that have a ring of ciliated tentacles around the mouth, called a lophophore. These were traditionally grouped together as the lophophorates. but it now appears that the lophophorate group may be paraphyletic, with some closer to the nemerteans and some to the molluscs and annelids. They include the Brachiopoda or lamp shells, which are prominent in the fossil record, the Entoprocta, the Phoronida, and possibly the Bryozoa or moss animals.
"""
Suggest a method to detect the nounchunks in a context


In [ ]:
# TODO: detect nounchunks dans un context : input (str)
# langue : anglais 
#nltk / spacy : comparaison 
# 1 - spacy
import spacy 
!python -m spacy download en_core_web_md 

2022-11-07 05:59:44.694678: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
nlp = spacy.load("en_core_web_md")
context = "The Lophotrochozoa, evolved within Protostomia, include two of the most successful animal phyla, the Mollusca and Annelida. The former, which is the second-largest animal phylum by number of described species, includes animals such as snails, clams, and squids, and the latter comprises the segmented worms, such as earthworms and leeches. These two groups have long been considered close relatives because of the common presence of trochophore larvae, but the annelids were considered closer to the arthropods because they are both segmented. Now, this is generally considered convergent evolution, owing to many morphological and genetic differences between the two phyla. The Lophotrochozoa also include the Nemertea or ribbon worms, the Sipuncula, and several phyla that have a ring of ciliated tentacles around the mouth, called a lophophore. These were traditionally grouped together as the lophophorates. but it now appears that the lophophorate group may be paraphyletic, with some closer to the nemerteans and some to the molluscs and annelids. They include the Brachiopoda or lamp shells, which are prominent in the fossil record, the Entoprocta, the Phoronida, and possibly the Bryozoa or moss animals. "
doc = nlp(context)
ents = set((ent.start, ent.end) for ent in doc.ents)
noun_chunks = [(100, nc.start, nc.end) for nc in doc.noun_chunks if (nc.start, nc.end) not in ents]
doc.ents = noun_chunks
for noun_chunk in doc.noun_chunks:
    print(noun_chunk.text, "\t", noun_chunk.start, "\t", 
          noun_chunk.end)
#problème : je ne peux pas avoir le nounchunk donné en exemple (à discuter pour améliorer)

The Lophotrochozoa 	 0 	 2
Protostomia 	 5 	 6
the most successful animal phyla 	 10 	 15
the Mollusca 	 16 	 18
Annelida 	 19 	 20
which 	 24 	 25
the second-largest animal phylum 	 26 	 32
number 	 33 	 34
described species 	 35 	 37
animals 	 39 	 40
snails 	 42 	 43
clams 	 44 	 45
squids 	 47 	 48
the segmented worms 	 53 	 56
earthworms 	 59 	 60
leeches 	 61 	 62
These two groups 	 63 	 66
close relatives 	 70 	 72
the common presence 	 74 	 77
trochophore larvae 	 78 	 80
the annelids 	 82 	 84
the arthropods 	 88 	 90
they 	 91 	 92
this 	 98 	 99
convergent evolution 	 102 	 104
many morphological and genetic differences 	 107 	 112
the two phyla 	 113 	 116
The Lophotrochozoa 	 117 	 119
the Nemertea or ribbon worms 	 121 	 126
the Sipuncula 	 127 	 129
several phyla 	 131 	 133
that 	 133 	 134
a ring 	 135 	 137
ciliated tentacles 	 138 	 140
the mouth 	 141 	 143
a lophophore 	 145 	 147
These 	 148 	 149
the lophophorates 	 154 	 156
it 	 158 	 159
the lophophorate group

**2e - TextBlob**

In [ ]:
import nltk
nltk.download('brown')
!pip install textblob
from textblob import TextBlob
context = "The Lophotrochozoa, evolved within Protostomia, include two of the most successful animal phyla, the Mollusca and Annelida. The former, which is the second-largest animal phylum by number of described species, includes animals such as snails, clams, and squids, and the latter comprises the segmented worms, such as earthworms and leeches. These two groups have long been considered close relatives because of the common presence of trochophore larvae, but the annelids were considered closer to the arthropods because they are both segmented. Now, this is generally considered convergent evolution, owing to many morphological and genetic differences between the two phyla. The Lophotrochozoa also include the Nemertea or ribbon worms, the Sipuncula, and several phyla that have a ring of ciliated tentacles around the mouth, called a lophophore. These were traditionally grouped together as the lophophorates. but it now appears that the lophophorate group may be paraphyletic, with some closer to the nemerteans and some to the molluscs and annelids. They include the Brachiopoda or lamp shells, which are prominent in the fossil record, the Entoprocta, the Phoronida, and possibly the Bryozoa or moss animals. "
blob = TextBlob(context)

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
print(blob.noun_phrases)

['lophotrochozoa', 'protostomia', 'successful animal phyla', 'mollusca', 'annelida', 'second-largest animal phylum', 'common presence', 'trochophore larvae', 'convergent evolution', 'genetic differences', 'lophotrochozoa', 'nemertea', 'ribbon worms', 'sipuncula', 'lophophorate group', 'brachiopoda', 'lamp shells', 'fossil record', 'entoprocta', 'phoronida', 'bryozoa', 'moss animals']


***COMPARAISON : SPACY A UN MEILLEUR RÉSULTAT MAIS TOUJOURS PAS TRÈS SATISFAISANT ***

Suggest a method which help us detect possible answers for a WHY question for the part in bold

In [ ]:
#TODO: suggest a method which help us detect possible answers for a WHY question

# première méthode : transformer 
!pip install transformers
from transformers import pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
nlp = pipeline("question-answering")

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [ ]:
context = "The Lophotrochozoa, evolved within Protostomia, include two of the most successful animal phyla, the Mollusca and Annelida. The former, which is the second-largest animal phylum by number of described species, includes animals such as snails, clams, and squids, and the latter comprises the segmented worms, such as earthworms and leeches. These two groups have long been considered close relatives because of the common presence of trochophore larvae, but the annelids were considered closer to the arthropods because they are both segmented. Now, this is generally considered convergent evolution, owing to many morphological and genetic differences between the two phyla. The Lophotrochozoa also include the Nemertea or ribbon worms, the Sipuncula, and several phyla that have a ring of ciliated tentacles around the mouth, called a lophophore. These were traditionally grouped together as the lophophorates. but it now appears that the lophophorate group may be paraphyletic, with some closer to the nemerteans and some to the molluscs and annelids. They include the Brachiopoda or lamp shells, which are prominent in the fossil record, the Entoprocta, the Phoronida, and possibly the Bryozoa or moss animals. "
result = nlp(question="Why the annelids were considered closer to the arthropods ?", context=context)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 8456.26it/s]


In [ ]:
print(f"Answer: '{result['answer']}', score: {round(result['score'], 3)}, start: {result['start']}, end: {result['end']}")

Answer: 'they are both segmented.', score: 0.446, start: 519, end: 543


**2e méthode AllenNLP**

In [ ]:
#AllenNLP (méthode sans entrainement) 
!pip install h5py
!pip install typing-extensions
!pip install wheel
!pip install -U spacy
!pip install allennlp
!pip install allennlp-models
from allennlp.predictors import Predictor
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bidaf-model-2020.03.19.tar.gz")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 6.7 MB/s 
     |████████████████████████████████| 490 kB 55.6 MB/s 
     |████████████████████████████████| 126 kB 55.3 MB/s 
     |████████████████████████████████| 806 kB 55.8 MB/s 
     |████████████████████████████████| 10.2 MB 26.7 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.6
    Uninstalling srsly-1.0.6:
      Successfully uninstalled srsly-1.0.6
  Attempting uninstall: preshed
    Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Attempti

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-3.3.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.2 MB)
  Using cached thinc-8.0.17-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (660 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.5
    Uninstalling thinc-8.1.5:
      Successfully uninstalled thinc-8.1.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.2
    Uninstalling spacy-3.4.2:
      Successfully uninstalled spacy-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.3.1 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
context = "The Lophotrochozoa, evolved within Protostomia, include two of the most successful animal phyla, the Mollusca and Annelida. The former, which is the second-largest animal phylum by number of described species, includes animals such as snails, clams, and squids, and the latter comprises the segmented worms, such as earthworms and leeches. These two groups have long been considered close relatives because of the common presence of trochophore larvae, but the annelids were considered closer to the arthropods because they are both segmented. Now, this is generally considered convergent evolution, owing to many morphological and genetic differences between the two phyla. The Lophotrochozoa also include the Nemertea or ribbon worms, the Sipuncula, and several phyla that have a ring of ciliated tentacles around the mouth, called a lophophore. These were traditionally grouped together as the lophophorates. but it now appears that the lophophorate group may be paraphyletic, with some closer to the nemerteans and some to the molluscs and annelids. They include the Brachiopoda or lamp shells, which are prominent in the fossil record, the Entoprocta, the Phoronida, and possibly the Bryozoa or moss animals. "

In [ ]:
answer=predictor.predict(
  passage=context,
  question= "Why the annelids were considered closer to the arthropods?"
)
answer['best_span_str']

'they are both segmented'